In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Models

In [2]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [3]:
experiment_name = "Covid-19 Cases"
seq_length = 14
forecast_horizon = 7
model_class = "PINPOINT_1CONV"

In [4]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [5]:
model

PINPOINT_1CONV(
  (main): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
    (1): Square()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=384, out_features=192, bias=True)
    (4): Linear(in_features=192, out_features=96, bias=True)
    (5): Linear(in_features=96, out_features=7, bias=True)
  )
)

# Dataset

In [6]:
# covid daily cases
df = pd.read_csv(f"{working_directory}/data/Covid19-italy.csv", parse_dates=["Date"], index_col="Date")
df = df.loc[:pd.Timestamp("2021-12-01"), 'Daily cases']
df.index.freq = 'D'
entire_ts = df

train = entire_ts.loc[:pd.Timestamp("2021-01-14")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]

plot_name = "Covid19"
yaxis_name = "Cases"

train = train.append(validation)

print(train)
print(test)

Date
2020-02-25       93
2020-02-26       78
2020-02-27      250
2020-02-28      238
2020-02-29      240
              ...  
2021-01-26    10593
2021-01-27    15204
2021-01-28    14372
2021-01-29    13574
2021-01-30    12715
Freq: D, Name: Daily cases, Length: 341, dtype: int64
Date
2021-01-31    11252
2021-02-01     7925
2021-02-02     9660
2021-02-03    13189
2021-02-04    13659
              ...  
2021-11-27    12877
2021-11-28    12932
2021-11-29     7975
2021-11-30    12764
2021-12-01    15085
Freq: D, Name: Daily cases, Length: 305, dtype: int64


# Expected outputs

In [7]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

expected_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [8]:
expected_output

Date
2021-01-31    10217.327148
2021-02-01     8715.881836
2021-02-02    11085.750977
2021-02-03    14822.344727
2021-02-04    15607.525391
                  ...     
2021-11-27    14281.905273
2021-11-28    11084.186523
2021-11-29     9543.678711
2021-11-30    12759.702148
2021-12-01    15842.121094
Freq: D, Length: 305, dtype: float64

In [9]:
print(experiment_name)
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

Covid-19 Cases
MAE of model PINPOINT_1CONV, forecast horizon: 7: 1063.07


## Encode the models

In [10]:
HE = Pyfhel()    
HE.contextGen(p=95337867, m=4096, intDigits=16, fracDigits=128) 
HE.keyGen()
HE.relinKeyGen(30, 3)

encoded_model = build_from_pytorch(HE, model.cpu().main)

# Encrypted processing

In [11]:
decrypted_output = None

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = inputs_normalized[-seq_length:].reshape(1, 1, seq_length)

    encrypted_input = encrypt_matrix(HE, inputs_normalized)

    for layer in encoded_model:
        encrypted_input = layer(encrypted_input)
    
    _min = HE.encodeFrac(scaler.min_[0])
    _scale = HE.encodeFrac(1.0 / scaler.scale_[0])
    
    encrypted_input = [list(map(lambda x: (x - _min)*_scale, encrypted_input[0]))]
    predict = decrypt_matrix(HE, encrypted_input)

    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

decrypted_output = pd.Series(data=forecast[:len(test)], index=test.index)

In [12]:
expected_output

Date
2021-01-31    10217.327148
2021-02-01     8715.881836
2021-02-02    11085.750977
2021-02-03    14822.344727
2021-02-04    15607.525391
                  ...     
2021-11-27    14281.905273
2021-11-28    11084.186523
2021-11-29     9543.678711
2021-11-30    12759.702148
2021-12-01    15842.121094
Freq: D, Length: 305, dtype: float64

In [13]:
decrypted_output

Date
2021-01-31    10417.379499
2021-02-01     8914.261679
2021-02-02    11285.248837
2021-02-03    14919.873183
2021-02-04    15660.975941
                  ...     
2021-11-27    14376.144759
2021-11-28    11283.234496
2021-11-29     9640.710943
2021-11-30    12860.674095
2021-12-01    15949.368750
Freq: D, Length: 305, dtype: float64

In [14]:
print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")
print(f"MAE of model {model} used on encrypted inputs, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, decrypted_output), 2)}")

MAE of model PINPOINT_1CONV, forecast horizon: 7: 1063.07
MAE of model PINPOINT_1CONV used on encrypted inputs, forecast horizon: 7: 1057.59


Difference between expected and obtained on encrypted data:

In [15]:
print(expected_output - decrypted_output)

Date
2021-01-31   -200.052350
2021-02-01   -198.379843
2021-02-02   -199.497860
2021-02-03    -97.528457
2021-02-04    -53.450551
                 ...    
2021-11-27    -94.239486
2021-11-28   -199.047973
2021-11-29    -97.032232
2021-11-30   -100.971947
2021-12-01   -107.247656
Freq: D, Length: 305, dtype: float64
